In [5]:
import itertools
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import(
    datasets, feature_extraction, model_selection, 
    pipeline, naive_bayes, metrics
)
import matplotlib.pyplot as plt

In [2]:
def extract_features(corpus):
    '''Extract TF-IDF features from corpus'''
    stop_words = nltk.corpus.stopwords.words('english')
    count_vectorizer = feature_extraction.text.CountVectorizer(
        tokenizer=nltk.word_tokenize,
        stop_words=stop_words,
        min_df=2, # The word must appear more than once
        # The higher ngram range is, the higher the vector space and computing cost
        ngram_range=(1, 2), # Allows for 1 and 2 word combinations
    )
    processed_corpus = count_vectorizer.fit_transform(corpus)
    # Can change params of TfidfTransformer if it lowers performance
    processed_corpus = feature_extraction.text.TfidfTransformer(
        ).fit_transform(processed_corpus)
    
    return processed_corpus

In [3]:
data_filepath = '20_news_groups/20_newsgroups'

newsgroups_data = datasets.load_files(
    data_filepath, shuffle=True, random_state=42, encoding='ISO-8859-1')
print(f'{len(newsgroups_data.data)} files loaded.')
print(f'They contain the following classes: {newsgroups_data.target_names}')
print()
print(newsgroups_data.data[0])

19997 files loaded.
They contain the following classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

Newsgroups: rec.sport.hockey
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!zaphod.mps.ohio-state.edu!uwm.edu!cs.utexas.edu!utnut!alchemy.chem.utoronto.ca!golchowy
From: golchowy@alchemy.chem.utoronto.ca (Gerald Olchowy)
Subject: Re: RUMOUR - Keenan signs with Rangers?
Message-ID: <1993Apr16.222232.17393@alchemy.chem.utoronto.ca>
Organization: University of Toronto Chemistry Department
References: <1993Apr16.171347.784@news.columbia.edu> <1993Apr16.183110.838@alchemy.chem.u

In [19]:
def test_validate_train_split(X, y,
                              test_size: float = None,
                              random_state: int = None, **kwargs
                              ) -> tuple:
    '''
        Completes Sci-Kit Learn's test_train_split twice to split data into
        three sections
        
        Paramaters:
        X: The dataset without the target present
        y: The target values of the dataset
        test_size: The proportion of the data set in the test set.
            It is also the proportion of the remainder used for the validation set
        random_state: Controls the shuffling 
        **kwargs: These are passed to both test_train_split functions
        
        Returns a tuple of:
            X_train, X_validate, X_test, y_train, y_validate, y_test
    '''
    # Complete test_train split
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, **kwargs)

    # Complete the test_validation split
    X_train, X_validate, y_train, y_validate = train_test_split(
        X_train_val, y_train_val, test_size=test_size,
        random_state=random_state, **kwargs)
    
    return X_train, X_validate, X_test, y_train, y_validate, y_test

In [21]:
# Complete a test_validate_train split
X_train, X_val, X_test, y_train, y_val, y_test = test_validate_train_split(
    newsgroups_data.data, newsgroups_data.target, test_size=0.2, random_state=42)

In [40]:
# Read in the stop words
stop_words = nltk.corpus.stopwords.words('english')
# Given out tokenizer there is an issue with contracted words from our stop_words
# you'd is in out stop_words, but is tokenised to ["you", "'d"]
# which would be missed, so run our stop words through the tokenizer to match
nested_tokenized_stop_words = [nltk.word_tokenize(stop_word) for stop_word in stop_words]
# This results in a list of lists which need to be flattened
tokenized_stop_words = [word
                        # for each list in the bigger list we want
                        for list_of_words in nested_tokenized_stop_words
                        # each word in the list
                        for word in list_of_words]

In [37]:
# Create the model pipeline
# Pipeline accepts a list of tuples, a name of a step, and a step
model = Pipeline([
    ('counts', feature_extraction.text.CountVectorizer(
        tokenizer=nltk.word_tokenize,
        min_df=2,
        ngram_range=(1,2),
        stop_words=tokenized_stop_words,
    )),
    ('tfidf', feature_extraction.text.TfidfTransformer()),
    ('naivebayes', naive_bayes.MultinomialNB()),
])

In [38]:
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(f'Accuracy of multinomial naive bayes = {np.mean(y_pred==y_val)}')

c:\Users\jsshe\Documents\learning\oreilly\Hands-on-NLP-with-NLTK-and-scikit-learn\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Accuracy of multinomial naive bayes = 0.8865625
